In [ ]:

# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, 
# along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all columns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

# Global lists to store results
BoosterVersion = []
LaunchSite = []
PayloadMass = []
Orbit = []
LaunchOutcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingOutcome = []


In [ ]:

# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            response = requests.get(f"https://api.spacexdata.com/v4/rockets/{x}").json()
            BoosterVersion.append(response['name'])


In [ ]:

# Function to get launch site details
def getLaunchSite(data):
    for launchpad_id in data['launchpad']:
        if launchpad_id:
            response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}").json()
            LaunchSite.append(response['name'])


In [ ]:

# Function to get payload mass and orbit details
def getPayloadData(data):
    for payload_id in data['payloads']:
        payload_list = payload_id
        for payload in payload_list:
            response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload}").json()
            PayloadMass.append(response.get('mass_kg', None))
            Orbit.append(response.get('orbit', None))


In [ ]:

# Function to get core landing outcome and reused details
def getCoreData(data):
    for core in data['cores']:
        core_list = core
        for core_item in core_list:
            core_response = requests.get(f"https://api.spacexdata.com/v4/cores/{core_item['core']}").json()
            Flights.append(core_response.get('flight', None))
            GridFins.append(core_item.get('gridfins', None))
            Reused.append(core_item.get('reused', None))
            Legs.append(core_item.get('legs', None))
            LandingOutcome.append(core_item.get('landing_success', None))


In [ ]:

# Reading the dataset and running functions
spacex_url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)
data = pd.json_normalize(response.json())

# Apply all functions to the dataset
getBoosterVersion(data)
getLaunchSite(data)
getPayloadData(data)
getCoreData(data)

# Adding extracted data to the dataset
data['BoosterVersion'] = BoosterVersion
data['LaunchSite'] = LaunchSite
data['PayloadMass'] = PayloadMass
data['Orbit'] = Orbit
data['Flights'] = Flights
data['GridFins'] = GridFins
data['Reused'] = Reused
data['Legs'] = Legs
data['LandingOutcome'] = LandingOutcome

# Save to CSV
data.to_csv('spacex_launch_data.csv', index=False)

# Display the first few rows of the dataframe
data.head()
